In [1]:
import torch
import torch.nn as nn
from typing import List, Dict, Tuple

## 참고 문헌 및 사이트:

### Transformer
* https://huggingface.co/transformers/model_doc/bert.html#overview
* https://paul-hyun.github.io/transformer-02/
* http://jalammar.github.io/illustrated-transformer/

### Pytorch
* https://pytorch.org/docs/stable/tensors.html
* https://pytorch.org/docs/master/generated/torch.nn.Embedding.html

### ETC
* https://yonghyuc.wordpress.com/2020/03/04/batch-norm-vs-layer-norm/
* https://baekyeongmin.github.io/dev/einsum/
* https://towardsdatascience.com/understanding-dimensions-in-pytorch-6edf9972d3be

# Self Attention

> 일반 attention을 설명할 때는 query, key, value에 대한 설명을 하는 포스트도 있고 아닌 포스트도 있어서 개념을 명확히 알기 어려웠다.  
하지만 이번 코드 구현을 통해 좀 더 자세하게 알 수 있는 시간이 되었ㄷ.

### Query, Key, Value
* query: attention 값을 이용 받을 대상자 -> 어디에 집중해야 할지 알고자 하는 대상
* key: attention 값을 만들때 사용되는 값 -> 어디에 집중해야 할까?
* softmax => $(query * key)/\sqrt{key vector dimension}$ 
* value: softmax와 연산이 이루어지는 대상

query, key, value는 결국 같은 차원을 갖게 되며(head dimension, head dimension),  
보통 query와 key's', value's'들이 함께 한 연산을 구성한다.    
그 이유는 생각해보면 attention을 원하는 단어는 하나이고, attention을 구성하는 단어는 문장 내 모든 단어이기 때문이다.  

### Self attention 절차:
1. encoder input vector(embedding of word)를 각기 $W^Q$, $W^K$, $W^V$와 곱한다.  
$W^Q$, $W^K$, $W^V$ 세 weight matrix 모두 훈련되어지는 행렬이다.

2. score 계산  
score는 집중도라고도 할 수 있으며, 문장 내에 어느 단어에 더 집중해야 하는 객관적 수치이다.  
score는 query와 key의 dot-product로 계산된다. 

3. score를 key vector 차원의 루트로 나눈다.  
이는 더 안정적인 gradients가 생성되는 것을 목표로 한다.

4. 문장에 있는 각 단어의 $(query * key)/\sqrt{key vector dimension}$ 로 산출된 값들에 대하여 softmax를 진행

5. 각 value vector와 softmax 값을 곱한다. 두 vector의 차원이 동일하므로 행렬 연산을 할 수 있다.  
만일 query가 집중을 해야하는 단어라면 높은 softmax value가 있을 것이다. 

6. 생성된 value vector를 모두 더한다!  

지금껏 살펴본 내용은 사실 하나의 query vector가 keys와 values들간의 상호작용이다.  
하지만 연산을 빠르게 하기 위해선 matrix calculation이 필요하다!

In [2]:
class SelfAttention(nn.Module):
    def __init__(self, embed_size, heads) -> None:
        # embed size가 256, heads가 8이라면 8*32로 나누어질 수 있다 -> 어떻게 나누어지는가?
        super(SelfAttention, self).__init__()
        self.embed_size = embed_size  # embed_size = 256
        self.heads = heads  # heads = 8
        self.head_dim = embed_size // heads  # head_dim = 32
        # 당연히 head_dim은 integer이므로 assert로 확인한다!

        assert (
            self.head_dim * heads == embed_size
        ), "Embed size needs to be div by heads"

        self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.fc_out = nn.Linear(heads * self.head_dim, embed_size)

        """
        * 차원을 head_dim(32)로 나누는 이유는 결국 나중에 Multihead attention으로 전환하기 위해서이다.
        * query / value / key는 모두 같은 차원
        * query = attention의 수혜자 (단어 하나)
        * keys = attention의 대상 (전체 단어 / 어디에 집중할 것이여!)
        * values = softmax 값을 얹는 대상
        * fc_out = heads들을 모두 concat하여 fc_out에 집어넣는다!
        """

    def forward(self, values, keys, query, mask) -> torch.tensor:
        N = query.shape[0]  # How many inputs we are going to send at the same time
        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]
        # These Three will be correspond to source / target sentence length
        # 단순히 차원수이며, src / trg에 비례하는 이유는 heads 갯수에 따라 달라질 거기 때문에

        # Split embedding into self.heads pieces
        values = values.reshape(N, value_len, self.heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.heads, self.head_dim)
        query = query.reshape(N, query_len, self.heads, self.head_dim)

        """
        values, keys, query는 모두 밑에서 out, 즉 word_embedding이 적용된 encoder_input이다.
        encoder_input의 꼴은 다음과 같다.
        [3,4,2,6,7,1,8,8,6,6],
        [36,11,264,3,2,5,3,5]
        ....
        여기에 word_embedding이 적용이 되면, |Batch_Size * Sequence_Length * Embedding_Dimension|
        
        query는 values / keys와 다르게 복수형이 아니다 -> 즉, 단일한 값이라는 거다.
        """

        values = self.values(values)
        keys = self.keys(keys)
        queries = self.queries(query)

        # |values| = Batch_Size * Sequence_Length * Heads * Head_Dim
        # |keys|   = Batch_Size * Sequence_Length * Heads * Head_Dim
        # |queries|= Batch_Size * Sequence_Length * Heads * Head_Dim

        energy = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])
        # n: batch_size
        # q: query_len
        # h: heads
        # d: heads dimension
        # k: key_len
        # einsum: bmm을 좀 더 쉽게 할 수 있다.
        # 차원을 알파벳으로 구성한 후 내가 원하는 차원의 Form으로 BMM으로 구성

        # |energy| = Batch_Size * Heads * Query_Length * Key_Length

        """
        matrix multiplication for various dimensions
        queries shape: (N, query_len, heads, heads_dim)
        keys shape: (N, key_len, heads, head_dim)
        energy shape: (N, heads, query_len, key_len)
        --> query_len: target / src sentence , key_len: src sentence
        query len이 얼만큼 key_len에 집중할 것인가?
        """
        if mask is not None:
            energy = energy.masked_fill(mask == 0, float("-1e20"))

        # encoder에 mask가 필요한지 여쭈어봐도 되겠습니까?
        # -> Pad index에 대하여 weights를 0으로 치환

        attention = torch.softmax(energy / (self.embed_size ** (1 / 2)), dim=3)
        # |attention| = Batch_Size * Heads * Query_Length * Key_Length

        # now multiply with value

        out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(
            N, query_len, self.heads * self.head_dim
        )
        # |out| = Batch_Size * Query_Length * Embedding_Size

        # attention shape: (N, heads, query_len, key_len)
        # values shape: (N, value_len, heads, heads_dim)
        # out return value = (N, query_len, heads, head_dim)

        # after einsum (N, query_len, heads, head_dim) then flatten last two dimensions

        out = self.fc_out(out)
        return out

# nn.Embedding Problem

nn.Embedding에 대한 오해가 있었다.  
마치 nn.Linear(a, b)에서 Linear의 input의 dimension이 a가 되어야 하는 것처럼,  
임베딩을 구축할때의 input도 a dimension의 one-hot embedding인 줄 알았다.

하지만 nn.Embedding(a, b)의 a는 단어의 차원이 아닌 갯수이므로 input의 차원은 sequence의 max_length가 되어야 한다.

### Embedding Example 1

In [3]:
# nn.Embedding return value example

embedding = nn.Embedding(10, 3)

input = torch.LongTensor([[1, 0, 0, 1], 
                          [1, 0, 1, 0]])

print(f"input shape ={input.shape}")

embeddings = embedding(input)
print(embeddings)
print(f"embedding shape = {embeddings.shape}")

input shape =torch.Size([2, 4])
tensor([[[-0.3801,  0.0379, -0.7233],
         [-0.8270, -0.9765, -0.8347],
         [-0.8270, -0.9765, -0.8347],
         [-0.3801,  0.0379, -0.7233]],

        [[-0.3801,  0.0379, -0.7233],
         [-0.8270, -0.9765, -0.8347],
         [-0.3801,  0.0379, -0.7233],
         [-0.8270, -0.9765, -0.8347]]], grad_fn=<EmbeddingBackward>)
embedding shape = torch.Size([2, 4, 3])


### Embedding Example 2

In [4]:
word_to_ix = {"hello": 0, "world": 1}
embeds = nn.Embedding(2, 5)  # 2 words in vocab, 5 dimensional embeddings
lookup_tensor = torch.tensor([word_to_ix["hello"]], dtype=torch.long)
hello_embed = embeds(lookup_tensor)
print(hello_embed)

tensor([[ 0.1962,  0.5557, -1.1133, -0.8332, -0.2471]],
       grad_fn=<EmbeddingBackward>)


위에서와 같이 input shape는 [2,4]이지만 nn.Embedding(10,3)에 문제 없이 input으로 역할을 할 수 있다.  
이는 input의 숫자들은 단어의 index이므로 숫자들이 10이내에만 존재하면 상관없게 된다.

그리고 return value의 shape는 torch.Size([2,4,3])인데 이것은,  
**(2개의 sequence)의 (4개의 단어)가 (3차원의 임베딩)으로 구성되었다** 를 의미한다.

## Transformer Blueprint

![blueprint](img/transformer_blueprint.png)

In [5]:
class TransformerBlock(nn.Module):
    def __init__(self, embed_size, heads, dropout, forward_expansion) -> None:
        super(TransformerBlock, self).__init__()
        self.attention = SelfAttention(embed_size, heads)
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)

        # Layernorm - BatchNorm 은 매우 유사
        # BatchNorm은 배치의 평균을 구한 뒤 Normalize
        # Layer는 각 예시의 평균을 구한 뒤 Normalize
        # LayerNorm이 Computation이 더 많다

        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, forward_expansion * embed_size),
            # forward_expansion - Node라는 개념과 연관, value = 4
            nn.ReLU(),
            nn.Linear(forward_expansion * embed_size, embed_size),
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, value, key, query, mask):
        attention = self.attention(value, key, query, mask)

        add_norm = self.norm1(attention + query)
        x = self.dropout(add_norm)

        forward = self.feed_forward(x)
        out = self.dropout(self.norm2(forward + x))
        return out

        # attention + query 자체가 skip connection
        # skip connection => 입력과 출력을 다음 레이어의 입력으로 보내는 것을 의미함

### Batch Norm vs Layer Norm
참고 사이트: https://yonghyuc.wordpress.com/2020/03/04/batch-norm-vs-layer-norm/


![batch & layer](./img/batch_layer_norm.png)

![transformer_input](./img/transformer_input.png){: width="50" height="100"}

### 의문: Self Attention과정은 하나의 Query와 Keys/Values들의 연산이다.
그런데 iteration도 없이 **한번에** 처리된다고 하는 것일까?


* Self-Attention은 하나의 query와 keys / values들의 연산이며, 이것은 Matrix Multiplication을 통해 진행하는 것이다. 
* 따라서 한번의 연산처럼 보이지만, 사실 여러 연산을 한번에 처리한 것과 같다.


In [6]:
x = torch.tensor([[1, 5, 6, 4, 3, 9, 5, 2, 0], [1, 8, 7, 3, 4, 5, 6, 7, 2]])

src_vocab_size = 10
embed_size = 256
word_embedding = nn.Embedding(src_vocab_size, embed_size)

In [7]:
word_embedding(x).shape

torch.Size([2, 9, 256])

In [8]:
class Encoder(nn.Module):
    def __init__(
        self,
        src_vocab_size,
        embed_size,
        num_layers,
        heads,
        device,
        forward_expansion,
        dropout,
        max_length,
        # related to positional embedding
        # positional embedding은 문장 길이에 영향을 받으므로, 문장의 최대 길이를 input으로 넣어줘야 한다.
    ):
        super(Encoder, self).__init__()
        self.embed_size = embed_size
        self.device = device
        self.word_embedding = nn.Embedding(src_vocab_size, embed_size)
        self.positional_embedding = nn.Embedding(max_length, embed_size)

        self.layers = nn.ModuleList(
            [
                TransformerBlock(
                    embed_size,
                    heads,
                    dropout=dropout,
                    forward_expansion=forward_expansion,
                )
                for _ in range(num_layers)
            ]
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        N, seq_length = x.shape
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)
        # range(a, b) = a부터 b까지
        # arange(a, b) = a부터 b-1까지
        # 왜 positional embedding에서 sinusidal function을 사용하지 않는 것이요?

        out = self.dropout(
            self.word_embedding(x) + self.positional_embedding(positions)
        )

        for layer in self.layers:
            out = layer(out, out, out, mask)

        return out

In [9]:
class DecoderBlock(nn.Module):
    def __init__(self, embed_size, heads, forward_expansion, dropout, device):
        super(DecoderBlock, self).__init__()

        self.attention = SelfAttention(embed_size, heads)
        self.norm = nn.LayerNorm(embed_size)
        self.transformer_block = TransformerBlock(
            embed_size, heads, dropout, forward_expansion
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, value, key, src_mask, trg_mask):
        # trg_mask는 필수이지만 src_mask는 선택이다.
        # src_mask는 <pad>에 대하여 masking을 하여 계산을 하지 않기 위해서이기 때문이다.

        attention = self.attention(x, x, x, trg_mask)
        query = self.dropout(self.norm(attention + x))
        out = self.transformer_block(value, key, query, src_mask)
        return out

In [10]:
class Decoder(nn.Module):
    def __init__(
        self,
        trg_vocab_size,
        embed_size,
        num_layers,
        heads,
        forward_expansion,
        dropout,
        device,
        max_length,
    ):
        super(Decoder, self).__init__()
        self.device = device
        self.word_embedding = nn.Embedding(trg_vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)
        self.layers = nn.ModuleList(
            [
                DecoderBlock(embed_size, heads, forward_expansion, dropout, device)
                for _ in range(num_layers)
            ]
        )
        self.fc_out = nn.Linear(embed_size, trg_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, enc_out, src_mask, trg_mask):
        N, seq_length = x.shape
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)
        x = self.dropout((self.word_embedding(x) + self.position_embedding(positions)))

        for layer in self.layers:
            x = layer(x, enc_out, enc_out, src_mask, trg_mask)
            # query, k, v이기 때문에 k,v는 enc_out에 포함

        out = self.fc_out(x)

        return out

In [11]:
class Transformer(nn.Module):
    def __init__(
        self,
        src_vocab_size,
        trg_vocab_size,
        src_pad_idx,
        trg_pad_idx,
        embed_size=256,
        num_layers=6,
        forward_expansion=4,
        heads=8,
        dropout=0,
        device="cuda",
        max_length=100,
    ):
        super(Transformer, self).__init__()

        self.encoder = Encoder(
            src_vocab_size,
            embed_size,
            num_layers,
            heads,
            device,
            forward_expansion,
            dropout,
            max_length,
        )

        self.decoder = Decoder(
            trg_vocab_size,
            embed_size,
            num_layers,
            heads,
            forward_expansion,
            dropout,
            device,
            max_length,
        )
        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device

    def make_src_mask(self, src):
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)
        # (N, 1, 1, src_len)
        return src_mask.to(self.device)

    def make_trg_mask(self, trg):
        N, trg_len = trg.shape
        trg_mask = torch.tril(torch.ones((trg_len, trg_len))).expand(
            N, 1, trg_len, trg_len
        )
        return trg_mask.to(self.device)

        # tril => triangular lower

    def forward(self, src, trg):
        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)
        enc_src = self.encoder(src, src_mask)
        out = self.decoder(trg, enc_src, src_mask, trg_mask)
        return out

# Small Example

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

x = torch.tensor([[1, 5, 6, 4, 3, 9, 5, 2, 0], [1, 8, 7, 3, 4, 5, 6, 7, 2]]).to(device)
trg = torch.tensor([[1, 7, 4, 3, 5, 9, 2, 0], [1, 5, 6, 2, 4, 7, 6, 2]]).to(device)

"""
<PAD> = 0
<SOS> = 1
<EOS> = 2

"""

src_pad_idx = 0
trg_pad_idx = 0
src_vocab_size = 10
trg_vocab_size = 10
model = Transformer(src_vocab_size, trg_vocab_size, src_pad_idx, trg_pad_idx).to(device)
out = model(x, trg[:, :-1])
print(out.shape)

torch.Size([2, 7, 10])


In [13]:
out

tensor([[[-1.7291e-01,  1.9720e-01, -2.0048e-01,  2.7124e-01, -8.4003e-01,
          -8.0453e-01,  1.1027e-01, -2.1203e-02,  1.0538e-02,  1.0009e+00],
         [ 4.8286e-01, -9.7255e-01, -5.4226e-01, -6.4622e-01,  2.8264e-01,
          -1.0968e+00,  3.4487e-01, -4.7285e-01, -2.3739e-01,  1.0901e+00],
         [ 1.3040e+00, -1.3005e-01, -3.8196e-01, -2.8422e-01, -2.5222e-01,
          -1.5123e+00, -7.1316e-01, -5.2477e-01,  1.1937e-01,  1.9800e-01],
         [-6.1350e-01, -4.9933e-01, -8.2925e-02,  3.0534e-01, -7.1628e-01,
          -3.7599e-01,  2.1025e-01, -1.2234e+00,  1.6607e-01, -4.0325e-02],
         [-2.1187e-01,  3.9327e-04,  3.0340e-01, -9.1456e-02, -9.4920e-01,
          -2.3883e-01, -4.1437e-01, -2.9551e-01, -2.8912e-01,  1.2098e-01],
         [ 4.6696e-01, -4.5956e-02, -5.4067e-01, -7.9009e-01, -2.3961e-01,
          -8.2380e-01,  2.6870e-01,  1.5595e-02, -6.1081e-01,  7.1001e-02],
         [ 1.6822e-01,  6.3791e-02,  4.4356e-01, -5.0223e-01,  5.7616e-01,
          -6.0673e-

In [16]:
out.size(0)

2

In [18]:
torch.arange(5).unsqueeze(1)

tensor([[0],
        [1],
        [2],
        [3],
        [4]])